# Core data utils

In [ ]:
#| default_exp models.utils

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export

# python
from typing import List, Optional, Dict, Any, Tuple

# torch
import torch
from torchvision.transforms import transforms
from torch.utils.data import random_split, Dataset, DataLoader
import torch.nn as nn

# lightning
from pytorch_lightning import LightningModule
from torchmetrics import Accuracy

## Model Module

In [ ]:
#| export
class ModelModule(LightningModule):
    def __init__(self,
                model:nn.Module, # pure pytorch model to be wrapped
                lr:float=1e-3 # learning rate
                ):

        super().__init__()
        self.save_hyperparameters(ignore=['model'])
        self.model = model
        self.loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.lr = lr

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        return optimizer
    
    def forward(self,
                x: torch.tensor, # X input images dim(B, H*W)
                ) -> torch.tensor: # y class probabilities (B, n_classes)
        return(self.model(x))

    def _step(self, batch:torch.tensor, batch_idx:int):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        acc = self.accuracy(y_hat, y)
        return loss, acc

    def training_step(self, batch:torch.tensor, batch_idx:int):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"train/loss": loss, "train/acc": acc}
        self.log_dict(metrics, on_epoch=True)
        return loss
    
    def validation_step(self, batch:torch.tensor, batch_idx:int, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"val/loss":loss, "val/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)
    
    def test_step(self, batch:torch.tensor, batch_idx:int, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"test/loss":loss, "test/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)

    def predict_step(self, batch:torch.tensor, batch_idx:int, dataloader_idx=0):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.model(x)
        return y_hat.argmax(dim=1)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()